In [ ]:
# https://spark.apache.org/docs/3.2.0/api/python/getting_started/index.html
#https://docs.databricks.com/delta/data-transformation/complex-types.html
# https://towardsdatascience.com/basics-of-apache-spark-configuration-settings-ca4faff40d45
# https://towardsdatascience.com/apache-spark-performance-boosting-e072a3ec1179
#C:\Users\PETERCHAPAGAIN\Downloads\vericred_ambetter_superior_health_plan_20210615.json
#https://github.com/lyhue1991/eat_pyspark_in_10_days
#https://www.datasciencemadesimple.com/set-difference-in-pyspark-difference-of-two-dataframe/
#pip install pyarrow
#spark.conf.set(“spark.sql.execution.arrow.enabled”, “true”)

In [1]:
# import os
# import sys
# os.environ["SPARK_HOME"] = "C:/Spark"  #/Users/anishchapagain/Spark
# os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.9.2-src.zip")
# sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Nested JSON File").getOrCreate()
print(spark.sparkContext.appName)

Nested JSON File


In [3]:
print(spark.sparkContext.appName)

Nested JSON File


In [ ]:
# len(df.head(5))>0 
# df.explain()

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [5]:
#path = "C:\Users\PETERCHAPAGAIN\Downloads\vericred_ambetter_superior_health_plan_20210615.json"
path = "C:/pyqt/data/imperial_health_plan.json"
df = spark.read.json(path=path)
df.count()

483

In [6]:
df.show(2)
df.printSchema()

+--------------------+------------------+---------------------+--------------------+--------------------+-------------+
|           addresses|group_affiliations|hospital_affiliations|            networks|            provider|  specialties|
+--------------------+------------------+---------------------+--------------------+--------------------+-------------+
|[{719 W COKE RD, ...|                []|                   []|[{Imperial Insura...|{null, CHRISTUS H...|[{Hospitals}]|
|[{1115 S SUNSET A...|                []|                   []|[{Imperial Health...|{null, EMANATE HE...|[{Hospitals}]|
+--------------------+------------------+---------------------+--------------------+--------------------+-------------+
only showing top 2 rows

root
 |-- addresses: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- address_string: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- languages: array (nullable = true)
 |    |   

In [7]:
df.columns

['addresses',
 'group_affiliations',
 'hospital_affiliations',
 'networks',
 'provider',
 'specialties']

In [8]:
df.schema.names

['addresses',
 'group_affiliations',
 'hospital_affiliations',
 'networks',
 'provider',
 'specialties']

In [9]:
df.provider.schema

Column<'provider[schema]'>

In [10]:
df.schema['addresses'].dataType

ArrayType(StructType(List(StructField(address_string,StringType,true),StructField(city,StringType,true),StructField(languages,ArrayType(StringType,true),true),StructField(office_name,StringType,true),StructField(phones,ArrayType(StructType(List(StructField(type,StringType,true),StructField(value,StringType,true))),true),true),StructField(state,StringType,true),StructField(street_line_1,StringType,true),StructField(street_line_2,StringType,true),StructField(zip,StringType,true))),true)

In [11]:
# Missing Values
#df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

In [12]:
# Null Values
#df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [13]:
df.groupby("provider.provider_type").count().show()

+-------------+-----+
|provider_type|count|
+-------------+-----+
|     facility|  483|
+-------------+-----+



In [14]:
df.groupby("provider.facility_type").count().show()

+-------------+-----+
|facility_type|count|
+-------------+-----+
|    Hospitals|  483|
+-------------+-----+



In [15]:
df.select("provider.unparsed_name").count() #site_uid

483

In [16]:
df.select("provider.unparsed_name").show(5) #site_uid
df.select("provider.unparsed_name").distinct().count() #site_uid

+--------------------+
|       unparsed_name|
+--------------------+
|CHRISTUS HEALTH M...|
|EMANATE HEALTH IN...|
|WICKENBURG COMMUN...|
|LOVELACE REGIONAL...|
|SAINT MARYS REG M...|
+--------------------+
only showing top 5 rows



230

In [102]:
distinctDF = df.distinct()
print("Distinct count: "+str(distinctDF.count()))
#distinctDF.show(2,truncate=True)

Distinct count: 483


In [17]:
df.select("provider.unparsed_name").show(5,truncate=False)

+---------------------------------------------------------+
|unparsed_name                                            |
+---------------------------------------------------------+
|CHRISTUS HEALTH MONTHER FRANCES HEALTH SYSTEM - WINNSBORO|
|EMANATE HEALTH INTER-COMMUNITY HOSPITAL                  |
|WICKENBURG COMMUNITY HOSPITAL                            |
|LOVELACE REGIONAL HOSPITAL - ROSWELL                     |
|SAINT MARYS REG MEDICAL CTR                              |
+---------------------------------------------------------+
only showing top 5 rows



In [18]:
df["provider"]

Column<'provider'>

### Filter, GroupBy

In [20]:
df.filter(df.provider.unparsed_name=="ST MARKS HOSPITAL").count()

3

In [21]:
df.filter(col("provider.unparsed_name")=="ST MARKS HOSPITAL").count() #reference using col()

3

In [22]:
df.groupBy(col("provider.unparsed_name")).count().show(5)

+--------------------+-----+
|       unparsed_name|count|
+--------------------+-----+
|HCA HOUSTON HEALT...|    2|
|CHRISTUS HEALTH GSHS|    6|
|BAYLOR SCOTT &amp...|    1|
|COMMUNITY FOUNDAT...|    3|
|SAN RAMON REGIONA...|    1|
+--------------------+-----+
only showing top 5 rows



In [23]:
#value_counts

#df.groupBy(col("provider.unparsed_name")).count().orderBy('count').show(10)
#df.groupBy(col("provider.unparsed_name")).count().orderBy('count',ascending=False).show(10,truncate=False)
df.groupBy(col("provider.unparsed_name")).count().orderBy(col('count').desc()).show(10,truncate=False)

+-----------------------------------------------------+-----+
|unparsed_name                                        |count|
+-----------------------------------------------------+-----+
|METHODIST MCKINNEY HOSPITAL LLC                      |20   |
|METHODIST HOSPITAL FOR SURGERY                       |16   |
|COMPLETE CARE MEDICAL ASSOCIATES                     |13   |
|UNKNOWN VENDOR                                       |13   |
|CALIFORNIA PACIFIC MEDICAL CENTER - CALIFORNIA CAMPUS|12   |
|METHODIST HOSPITALS OF DALLAS                        |12   |
|SUTTER MEDICAL CENTER SACRAMENTO                     |11   |
|NORTH AUSTIN MEDICAL CENTER                          |11   |
|METHODIST HOSPITAL                                   |9    |
|BANNER BEHAVIOR HEALTH                               |8    |
+-----------------------------------------------------+-----+
only showing top 10 rows



In [25]:
#df.filter(col("provider.unparsed_name").like("%CALIFORNIA%")).show()
df.filter(col("provider.unparsed_name").rlike("^SCOTT.*")).select(['addresses','networks','provider','specialties']).show(truncate=False)
df.where(col("provider.unparsed_name").rlike("^SCOTT")).count()

+--------------------+--------------------+--------------------+-------------+
|           addresses|            networks|            provider|  specialties|
+--------------------+--------------------+--------------------+-------------+
|[{7400 E THOMPSON...|[{Imperial Insura...|{null, SCOTTSDALE...|[{Hospitals}]|
|[{3535 N SCOTTSDA...|[{Imperial Insura...|{null, SCOTTSDALE...|[{Hospitals}]|
|[{7400 E OSBORN R...|[{Imperial Insura...|{null, SCOTTSDALE...|[{Hospitals}]|
|[{10460 N 92ND ST...|[{Imperial Insura...|{null, SCOTTSDALE...|[{Hospitals}]|
|[{9003 E SHEA BLV...|[{Imperial Insura...|{null, SCOTTSDALE...|[{Hospitals}]|
+--------------------+--------------------+--------------------+-------------+



5

### Specialities

In [26]:
specs = df.select(explode(df.specialties)).select('col.name') #col: API name
specs.show(3) 
specs.printSchema()

+---------+
|     name|
+---------+
|Hospitals|
|Hospitals|
|Hospitals|
+---------+
only showing top 3 rows

root
 |-- name: string (nullable = true)



In [27]:
specs.groupBy(col("name")).count().orderBy(col('count').desc()).show(10,truncate=False)

+---------+-----+
|name     |count|
+---------+-----+
|Hospitals|483  |
+---------+-----+



### Networks

In [28]:
df.select("networks").printSchema()

root
 |-- networks: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- tier: string (nullable = true)



In [29]:
networks = df.select(explode(df.networks)).select('col.name','col.tier') #col: API name
networks.show(3) 

+--------------------+----+
|                name|tier|
+--------------------+----+
|Imperial Insuranc...|    |
|Imperial Health P...|    |
|Imperial Insuranc...|    |
+--------------------+----+
only showing top 3 rows



In [30]:
networks.groupBy(col("name")).count().orderBy(col('count').desc()).show(10,truncate=False) #value_count

+----------------------------------+-----+
|name                              |count|
+----------------------------------+-----+
|Imperial Insurance Companies      |362  |
|Imperial Health Plan of California|121  |
+----------------------------------+-----+



### Affiliations

In [101]:
df.select("hospital_affiliations")

DataFrame[hospital_affiliations: array<string>]

In [64]:
df.hospital_affiliations

Column<'hospital_affiliations'>

In [ ]:
hospitals = df.select(explode(df.hospital_affiliations)).select('col.name') #col: API name
networks.show(3) 

In [31]:
# groups = 
df.select("group_affiliations").printSchema()

root
 |-- group_affiliations: array (nullable = true)
 |    |-- element: string (containsNull = true)



### Addresses

In [33]:
df.select('addresses').printSchema()
df.schema.names

root
 |-- addresses: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- address_string: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- languages: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- office_name: string (nullable = true)
 |    |    |-- phones: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |    |    |-- value: string (nullable = true)
 |    |    |-- state: string (nullable = true)
 |    |    |-- street_line_1: string (nullable = true)
 |    |    |-- street_line_2: string (nullable = true)
 |    |    |-- zip: string (nullable = true)



['addresses',
 'group_affiliations',
 'hospital_affiliations',
 'networks',
 'provider',
 'specialties']

In [112]:
for column_name in df.schema.names:
        print("Outside isinstance loop: " + column_name)

Outside isinstance loop: addresses
Outside isinstance loop: group_affiliations
Outside isinstance loop: hospital_affiliations
Outside isinstance loop: networks
Outside isinstance loop: provider
Outside isinstance loop: specialties


In [152]:
df_address = df.withColumn("address",explode("addresses")).drop("addresses")

In [153]:
df_networks = df_address.withColumn("network",explode("networks")).drop("networks")

In [165]:
dfnew = df_networks.withColumn("speciality",explode("specialties")).drop("specialties")

In [162]:
# df_group = df_hos.withColumn("hospitals",explode("hospital_affiliations")).drop("hospital_affiliations")

In [163]:
# df_new = df_group.withColumn("groups",explode("group_affiliations")).drop("group_affiliations")

In [167]:
dfnew.show(2)

+------------------+---------------------+--------------------+--------------------+--------------------+-----------+
|group_affiliations|hospital_affiliations|            provider|             address|             network| speciality|
+------------------+---------------------+--------------------+--------------------+--------------------+-----------+
|                []|                   []|{null, CHRISTUS H...|{719 W COKE RD, W...|{Imperial Insuran...|{Hospitals}|
|                []|                   []|{null, EMANATE HE...|{1115 S SUNSET AV...|{Imperial Health ...|{Hospitals}|
+------------------+---------------------+--------------------+--------------------+--------------------+-----------+
only showing top 2 rows



In [168]:
dfnew.printSchema()

root
 |-- group_affiliations: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- hospital_affiliations: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- provider: struct (nullable = true)
 |    |-- accepting_new_patients: string (nullable = true)
 |    |-- facility_name: string (nullable = true)
 |    |-- facility_type: string (nullable = true)
 |    |-- first_name: string (nullable = true)
 |    |-- gender: string (nullable = true)
 |    |-- last_name: string (nullable = true)
 |    |-- license_number: string (nullable = true)
 |    |-- middle_name: string (nullable = true)
 |    |-- npi: string (nullable = true)
 |    |-- pcp: string (nullable = true)
 |    |-- pcp_id: string (nullable = true)
 |    |-- provider_type: string (nullable = true)
 |    |-- rating: struct (nullable = true)
 |    |    |-- scale: string (nullable = true)
 |    |    |-- score: string (nullable = true)
 |    |-- site_uid: string (nullable = true)
 |    |--

In [184]:
dfnew.select("address.zip").distinct().count()

269

In [173]:
dfnew.groupBy(col("address.zip")).count().orderBy(col('count').desc()).show(10,truncate=False) #value_count

+-----+-----+
|zip  |count|
+-----+-----+
|78216|13   |
|78229|10   |
|85251|10   |
|79902|9    |
|95816|8    |
|78258|7    |
|75070|7    |
|75001|6    |
|85206|6    |
|85006|6    |
+-----+-----+
only showing top 10 rows



In [174]:
dfnew.groupBy(col("address.state")).count().orderBy(col('count').desc()).show(10,truncate=False) #value_count

+-----+-----+
|state|count|
+-----+-----+
|TX   |263  |
|CA   |121  |
|AZ   |84   |
|UT   |10   |
|NM   |3    |
|NV   |2    |
+-----+-----+



In [180]:
dfnew.groupBy("address.city","address.state").count().orderBy(col('count').desc()).show(10,truncate=False) #value_count

+-------------+-----+-----+
|city         |state|count|
+-------------+-----+-----+
|SAN ANTONIO  |TX   |48   |
|HOUSTON      |TX   |20   |
|EL PASO      |TX   |19   |
|AUSTIN       |TX   |18   |
|DALLAS       |TX   |18   |
|TUCSON       |AZ   |15   |
|SCOTTSDALE   |AZ   |13   |
|SAN FRANCISCO|CA   |13   |
|PHOENIX      |AZ   |12   |
|MCKINNEY     |TX   |12   |
+-------------+-----+-----+
only showing top 10 rows



In [183]:
dfnew.groupBy("address.city","address.state","address.zip").count().orderBy(col('count').desc()).show(10,truncate=False) #value_count

+-----------+-----+-----+-----+
|city       |state|zip  |count|
+-----------+-----+-----+-----+
|SAN ANTONIO|TX   |78216|13   |
|SAN ANTONIO|TX   |78229|10   |
|SCOTTSDALE |AZ   |85251|10   |
|EL PASO    |TX   |79902|9    |
|SACRAMENTO |CA   |95816|8    |
|MCKINNEY   |TX   |75070|7    |
|SAN ANTONIO|TX   |78258|7    |
|MESA       |AZ   |85206|6    |
|PHOENIX    |AZ   |85006|6    |
|TUCSON     |AZ   |85719|6    |
+-----------+-----+-----+-----+
only showing top 10 rows



In [203]:
dfnew.filter(col("provider.unparsed_name")=="METHODIST MCKINNEY HOSPITAL LLC").select('address.*').show(truncate=True)

+--------------------+----------+---------+-----------+--------------------+-----+--------------------+-------------+-----+
|      address_string|      city|languages|office_name|              phones|state|       street_line_1|street_line_2|  zip|
+--------------------+----------+---------+-----------+--------------------+-----+--------------------+-------------+-----+
|4713 HWY 121, STE...|THE COLONY|       []|           |[{phone, (469) 36...|   TX|4713 HWY 121, STE...|             |75056|
|4713 HWY 121, STE...|THE COLONY|       []|           |[{phone, (469) 36...|   TX|4713 HWY 121, STE...|             |75056|
|6501 PRESTON ROAD...|     PLANO|       []|           |[{phone, (972) 56...|   TX|6501 PRESTON ROAD...|             |75024|
|3501 MIDWAY ROAD,...|     PLANO|       []|           |[{phone, (972) 78...|   TX|3501 MIDWAY ROAD,...|             |75093|
|6501 PRESTON ROAD...|     PLANO|       []|           |[{phone, (972) 56...|   TX|6501 PRESTON ROAD...|             |75024|
|3501 MI

In [208]:
dfnew.filter(col("provider.unparsed_name")=="METHODIST MCKINNEY HOSPITAL LLC")\
.select('address.address_string','address.phones','speciality.name','network.name')\
.show(truncate=False)

+---------------------------------------------------------+---------------------------+---------+----------------------------+
|address_string                                           |phones                     |name     |name                        |
+---------------------------------------------------------+---------------------------+---------+----------------------------+
|4713 HWY 121, STE 306, THE COLONY, TX 75056              |[{phone, (469) 362 - 2607}]|Hospitals|Imperial Insurance Companies|
|4713 HWY 121, STE 306, THE COLONY, TX 75056              |[{phone, (469) 362 - 2607}]|Hospitals|Imperial Insurance Companies|
|6501 PRESTON ROAD, STE 102, PLANO, TX 75024              |[{phone, (972) 569 - 2700}]|Hospitals|Imperial Insurance Companies|
|3501 MIDWAY ROAD, STE 198, PLANO, TX 75093               |[{phone, (972) 781 - 2322}]|Hospitals|Imperial Insurance Companies|
|6501 PRESTON ROAD, STE 102, PLANO, TX 75024              |[{phone, (972) 569 - 2700}]|Hospitals|Imperial Insur

In [ ]:
dfnew.filter(col("provider.unparsed_name")=="METHODIST MCKINNEY HOSPITAL LLC")\
.select('address.address_string','address.phones','speciality.name','network.name')\
.distinct()